<a href="https://colab.research.google.com/github/Selma266/Malicious-URL-Detection/blob/main/Malicious_URL_Detection_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Chargement Phishing URLs dataset + prétraitement

In [2]:
import pandas as pd #la bibliothèque Pandas, utilisée pour la manipulation et l’analyse de données tabulaires.
df = pd.read_csv("/content/drive/MyDrive/Phishing URLs_cleaned.csv")  # read dataset url,label  #Chargement du dataset
df.head()

,url,label
0,https://docs.google.com/presentation/d/e/2PACX...,malicious
1,https://btttelecommunniccatiion.weeblysite.com/,malicious
2,https://kq0hgp.webwave.dev/,malicious
3,https://brittishtele1bt-69836.getresponsesite....,malicious
4,https://bt-internet-105056.weeblysite.com/,malicious


Suppression des lignes redondantes

In [3]:
# BEFORE removing duplicates
print("Before removing duplicates:")
print("Shape:", df.shape)

# Step 2: Remove duplicate rows
df = df.drop_duplicates()   #Suppression des doublons

# AFTER removing duplicates
print("\nAfter removing duplicates:")
print("Shape:", df.shape)

Before removing duplicates:
Shape: (504983, 2)

After removing duplicates:
Shape: (504933, 2)


Supprimer les NaN+ Conversion label en valeur Numeriques

In [4]:
# Supprimer les lignes où url ou label est NaN
df = df.dropna(subset=["url", "label"]).reset_index(drop=True)  #reset_index(drop=True) Réinitialise l’index du DataFrame
# strip() supprime les espaces au début et à la fin URL
df["url"] = df["url"].str.strip()
# Conversion des labels texte en valeurs numériques
df["label"] = df["label"].map({"benign": 0, "malicious": 1}).astype(int) #Encodage des labels
print(df["label"].value_counts()) #Vérification de la distribution des classes Afichages

label
0    345738
1    159195
Name: count, dtype: int64


Creation des 3 distributions dataset1(50% 50%),dataset2(90% 10%)et dataset(10%-90%)

In [5]:
#Génère les trois distributions expérimentales à partir du même dataset source.
import pandas as pd
# Sépare le dataset en deux sous-ensembles selon la classe (bénigne / malveillante).
benign = df[df['label'] == 0]
malicious = df[df['label'] == 1]
#Affiche le nombre d’échantillons par classe pour vérifier la disponibilité des données.
print(f"URLs bénignes disponibles: {len(benign)}")
print(f"URLs malveillantes disponibles: {len(malicious)}")
print(f"Total URLs disponibles: {len(df)}")

# Taille fixe pour tous les datasets\ to avoid 'Cannot take a larger sample than population error
TOTAL_SIZE = 176883

# ===== Dataset 1: Équilibré (50-50) =====
def create_balanced_dataset(benign, malicious, total_size=176883):
    """
    Crée un dataset équilibré avec 50% benign et 50% malicious
    """
    size_per_class = total_size // 2  #  176883 // 2 = 88441 pour chaque classe

    ben_sample = benign.sample(n=size_per_class, random_state=42)
    mal_sample = malicious.sample(n=size_per_class, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample]) # Concatène  les deux DataFrames
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Dataset 2: 90% bénignes, 10% malveillantes =====
def create_imbalanced_dataset_90_10(benign, malicious, total_size=176883):
    """
    Crée un dataset avec 90% benign et 10% malicious
    """
    ben_size = int(total_size * 0.9)
    mal_size = int(total_size * 0.1)

    ben_sample = benign.sample(n=ben_size, random_state=42)
    mal_sample = malicious.sample(n=mal_size, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample])
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Dataset 3: 10% bénignes, 90% malveillantes =====
def create_imbalanced_dataset_10_90(benign, malicious, total_size=176883):
    """
    Crée un dataset avec 10% benign et 90% malicious
    """
    ben_size = int(total_size * 0.1)
    mal_size = int(total_size * 0.9)

    ben_sample = benign.sample(n=ben_size, random_state=42)
    mal_sample = malicious.sample(n=mal_size, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample])
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Créer les 3 datasets =====
print("\n" + "="*60)
print("CRÉATION DES DATASETS")
print("="*60)

dataset_balanced = create_balanced_dataset(benign, malicious, TOTAL_SIZE)
dataset_imb_90_10 = create_imbalanced_dataset_90_10(benign, malicious, TOTAL_SIZE)
dataset_imb_10_90 = create_imbalanced_dataset_10_90(benign, malicious, TOTAL_SIZE)

# ===== Afficher les statistiques =====
print("\n📊 DATASET 1 - BALANCED (50-50):")
print(f"   Total samples: {len(dataset_balanced)}")
print(f"   Distribution:")
print(dataset_balanced['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_balanced['label'].value_counts(normalize=True).sort_index() * 100)

print("\n📊 DATASET 2 - IMBALANCED (90% benign - 10% malicious):")
print(f"   Total samples: {len(dataset_imb_90_10)}")
print(f"   Distribution:")
print(dataset_imb_90_10['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_imb_90_10['label'].value_counts(normalize=True).sort_index() * 100)

print("\n📊 DATASET 3 - IMBALANCED (10% benign - 90% malicious):")
print(f"   Total samples: {len(dataset_imb_10_90)}")
print(f"   Distribution:")
print(dataset_imb_10_90['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_imb_10_90['label'].value_counts(normalize=True).sort_index() * 100)


URLs bénignes disponibles: 345738
URLs malveillantes disponibles: 159195
Total URLs disponibles: 504933

CRÉATION DES DATASETS

📊 DATASET 1 - BALANCED (50-50):
   Total samples: 176882
   Distribution:
label
0    88441
1    88441
Name: count, dtype: int64
   Pourcentages:
label
0    50.0
1    50.0
Name: proportion, dtype: float64

📊 DATASET 2 - IMBALANCED (90% benign - 10% malicious):
   Total samples: 176882
   Distribution:
label
0    159194
1     17688
Name: count, dtype: int64
   Pourcentages:
label
0    90.000113
1     9.999887
Name: proportion, dtype: float64

📊 DATASET 3 - IMBALANCED (10% benign - 90% malicious):
   Total samples: 176882
   Distribution:
label
0     17688
1    159194
Name: count, dtype: int64
   Pourcentages:
label
0     9.999887
1    90.000113
Name: proportion, dtype: float64


Charge le tokenizer RoBERTa pré-entraîné

In [6]:
!pip install datasets

In [7]:
import torch
from datasets import Dataset
from transformers import RobertaTokenizerFast

In [8]:
#Charge le tokenizer RoBERTa pré-entraîné, qui convertit les URLs en tokens numériques exploitables par le modèle.
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")  #Ce bloc charge le tokenizer RoBERTa, mais pas le modèle lui-même.
#Confirme le chargement correct du tokenizer et affiche la taille de son vocabulaire.
print("Tokenizer chargé avec succès!")
print(f"Taille du vocabulaire: {tokenizer.vocab_size}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Tokenizer chargé avec succès!
Taille du vocabulaire: 50265


Définir la fonction de tokenisation

In [9]:
# ÉTAPE 2 : Définir la fonction de tokenisation pour tokeniser un batch de données
def tokenize_batch(batch):
    return tokenizer(
        batch["url"],#le texte (URLs) à convertir en tokens
        truncation=True,  #Si le texte est trop long, on ne garde que les 128 premiers tokens
        padding="max_length",  #padding="max_length" Tous les textes doivent avoir la même longueur+remplit les textes courts jusqu’à max_length
        max_length=128 #limite les séquences à 128 tokens
    )

 convertir les 3 dataFrames pandas en HuggingFace Dataset, RoBERTa + Trainer ne travaillent pas directement avec pandas.

In [10]:
#Conversion des datasets Pandas
hf_balanced = Dataset.from_pandas(dataset_balanced)
hf_90_10 = Dataset.from_pandas(dataset_imb_90_10)  #(90% benign - 10% malicious)
hf_10_90 = Dataset.from_pandas(dataset_imb_10_90)  #(10% benign - 90% malicious)

Appliquer le tokenizer RoBERTa aux 3 datasets, Obtenir des datasets prêts pour l’entraînement

In [11]:
# Tokenisation du dataset équilibré (50–50)
hf_balanced = hf_balanced.map(
    tokenize_batch,#Applique la fonction tokenize_batch en batch sur le dataset Hugging Face.
    batched=True,#Traiter les données par lots (ex: 1000 lignes à la fois) au lieu d'une par une Taille du batch par défaut : 1000 lignes/ traitement vectorisé, plus rapide pour les grands datasets.
    remove_columns=["url"]# le modèle n’a besoin que des input_ids et attention_mask. on garde seulement les features nécessaires
)
#Taille du batch non précisée → Hugging Face utilise 1000 par défaut, mais peut être ajustée si GPU limité.

Map:   0%|          | 0/176882 [00:00<?, ? examples/s]

In [12]:
#Tokenisation du dataset 10–90
hf_10_90 = hf_10_90.map(
    tokenize_batch,
    batched=True,
    remove_columns=["url"]
)

Map:   0%|          | 0/176882 [00:00<?, ? examples/s]

In [13]:
#Tokenisation du dataset 90–10
hf_90_10 = hf_90_10.map(
    tokenize_batch,
    batched=True,
    remove_columns=["url"]
)

Map:   0%|          | 0/176882 [00:00<?, ? examples/s]

Définir le format PyTorch:Convertit les datasets HF en tenseurs PyTorch

In [14]:
columns = ["input_ids", "attention_mask", "label"] #Spécifie les colonnes que le modèle RoBERTa utilisera pour l’entraînement
#Convertit les datasets HF en tenseurs PyTorch, prêts pour l’entraînement,  Ne garde que les colonnes spécifiées (input_ids, attention_mask, label) → optimisation mémoire.
hf_balanced.set_format(type="torch", columns=columns)
hf_90_10.set_format(type="torch", columns=columns)
hf_10_90.set_format(type="torch", columns=columns)

Affichage pour faire la vérification

In [15]:
print(hf_balanced)
print(hf_90_10)
print(hf_10_90)

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 176882
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 176882
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 176882
})


Séparation des données (TRAIN / TEST (80%-20%)) pour les 3 dataset

In [16]:
#Sépare le dataset Hugging Face en train (80%) et test (20%) dataset 50%-50%
split_balanced = hf_balanced.train_test_split(test_size=0.2, seed=42)
train_balanced = split_balanced["train"]
test_balanced = split_balanced["test"]

# Vérification affiche le nombre d’échantillons pour train et test afin de vérifier le split.
print("Balanced Dataset")
print("Train size:", len(train_balanced))
print("Test size :", len(test_balanced))

Balanced Dataset
Train size: 141505
Test size : 35377


In [17]:
# dataset 90%-10%
split_90_10 = hf_90_10.train_test_split(test_size=0.2, seed=42)
train_90_10 = split_90_10["train"]
test_90_10 = split_90_10["test"]

# dataset 10%-90%
split_10_90 = hf_10_90.train_test_split(test_size=0.2, seed=42)
train_10_90 = split_10_90["train"]
test_10_90 = split_10_90["test"]
#Vérification similaire pour chacun (taille + distribution)

FINE-TUNING AVEC LoRA

Si tu entraînes le modèle sur balanced puis tu ré-entraîne sur 90/10 avec le même objet model, tu “contamines” l’expérience (carry-over).

➡️ Tu dois recréer un modèle neuf (mêmes poids init) pour chaque dataset.

In [ ]:
from transformers import RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model #outils pour appliquer LoRA
#Tokenizer:convertit le texte en tokens (input_ids, attention_mask) deja charger, On charge le modèle RoBERTa pour la classification binaire il reçoit ces tokens et produit des prédictions/ classification

# Charger modèle pré-entraîné
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2 #Définit num_labels=2 pour la classification binaire (benign/malicious).
)
#la taille du modèle est grande (~125M paramètres) → GPU nécessaire pour fine-tuning complet, LoRA réduit ce coût.

#Paramètres standard pour fine-tuning efficace avec LoRA (Configuration LoRA )
lora_config = LoraConfig(
    r=8, #rang de la décomposition low-rank → contrôle la capacité d’adaptation
    lora_alpha=16,   #facteur d’échelle → stabilise l’entraînement
    lora_dropout=0.1,  #régularisation pour éviter l’overfitting
    target_modules=["query", "value"], #appliqué uniquement aux matrices Q et V des transformers
    bias="none",
    task_type="SEQ_CLS"  # Sequence Classification, tâche de classification de séquences
)

#  Appliquer LoRA au modèle,Transforme le modèle RoBERTa en modèle LoRA, où seules certaines couches sont entraînables.
model = get_peft_model(model, lora_config)

In [18]:
from transformers import RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType #outils pour appliquer LoRA
#Tokenizer:convertit le texte en tokens (input_ids, attention_mask) deja charger, On charge le modèle RoBERTa pour la classification binaire il reçoit ces tokens et produit des prédictions/ classification

def build_lora_roberta():
# Charger modèle pré-entraîné
    base = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2) #Définit num_labels=2 pour la classification binaire (benign/malicious).
#la taille du modèle est grande (~125M paramètres) → GPU nécessaire pour fine-tuning complet, LoRA réduit ce coût.
#Paramètres standard pour fine-tuning efficace avec LoRA (Configuration LoRA )
    lora_config = LoraConfig(
        r=8, #rang de la décomposition low-rank → contrôle la capacité d’adaptation
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["query", "value"],#appliqué uniquement aux matrices Q et V des transformers
        bias="none",
        task_type=TaskType.SEQ_CLS # Sequence Classification, tâche de classification de séquences
    )
    #  Appliquer LoRA au modèle,Transforme le modèle RoBERTa en modèle LoRA, où seules certaines couches sont entraînables.
    model = get_peft_model(base, lora_config)

    return model

Préparer l’entraînement

In [19]:
from transformers import TrainingArguments
from transformers import Trainer

In [20]:
# TrainingArguments : classe qui permet de définir tous les paramètres d’entraînement du modèle, Nombre d’epochs, taille des batches, learning rate, Stratégie d’évaluation et de sauvegarde GPU/FP16, logging, etc.
#Trainer : classe qui gère automatiquement l’entraînement, l’évaluation et la sauvegarde du modèle, automatisera tout l’entraînement : forward/backward pass, calcul de loss, optimisation, scheduler, checkpointing.
#Trainer appelle automatiquement  fonction compte metric à chaque évaluation (eval_dataset).
#output_dir doit être différent pour chaque expérience,Sinon tu écrases les résultats du balanced avec ceux du 90/10 etc.
training_args = TrainingArguments(
    output_dir="./results/balanced", # dossier pour sauvegarder les checkpoints
    num_train_epochs=1, #nombre de passages sur tout le dataset
    per_device_train_batch_size=16, # batch size pour l'entraînement
    per_device_eval_batch_size=32,# batch size pour évaluation
    learning_rate=5e-5,
    save_strategy="epoch", # sauvegarder modèle à chaque epoch
    eval_strategy="epoch", # Added evaluation strategy , évaluer à chaque fin d'epoch
    fp16=True  # activer float16 si GPU compatible
)

Fonction d’évaluation

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

In [22]:
#calculer des métriques personnalisées (precision, recall, F1-score) pendant l’évaluation.
def compute_metrics(eval_pred):
    """
    Cette fonction est appelée par Trainer à chaque évaluation.
    eval_pred: tuple (logits, labels)
    Retourne: dictionnaire {"precision":..., "recall":..., "f1":...}
    """
    #Entrée : un tuple (logits, labels), Sortie : dictionnaire avec les métriques, compatible avec HF Trainer.
    logits, labels = eval_pred
    # Convertir logits en prédictions de classe
    preds = torch.argmax(torch.tensor(logits), axis=1).numpy()
    labels = labels.astype(int)

    # Calcul des métriques
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [23]:
import torch
print(torch.cuda.is_available())  #entraînement sur GPU


True


In [24]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))  #Vérifier le GPU


#|# Matériel | Temps attendu (1 epoch) |
#| --------  | ----------------------- |
#| CPU      |  20–100 h                |
#| GPU T4   | ~30–60 min               |
#| GPU V100 | ~20–40 min               |
#| GPU A100 | ~5–10 min                |



True
Tesla T4


Créer Trainer+ Lancer l'entraînement balanced dataset+calcul les metric

In [25]:

# IMPORTANT: nouveau modèle pour chaque expérience
model_balanced = build_lora_roberta()
#Créer Trainer
trainer_balanced = Trainer(
   model= model_balanced,
    args=training_args,
    train_dataset=train_balanced, #  avec dataset équilibré
    eval_dataset=test_balanced,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
# Lancer l'entraînement
trainer_balanced.train() # entraîne le modèle en appliquant LoRA sur les couches ciblées

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1699397627.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_balanced = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.030200,0.044092,0.999029,0.986414,0.992682


TrainOutput(global_step=8845, training_loss=0.046556216488455975, metrics={'train_runtime': 789.1514, 'train_samples_per_second': 179.313, 'train_steps_per_second': 11.208, 'total_flos': 9404282506644480.0, 'train_loss': 0.046556216488455975, 'epoch': 1.0})

In [26]:
#évalue le modèle sur le dataset de test. Retourne un dictionnaire avec loss et métriques ( precision, recall, F1).
metrics = trainer_balanced.evaluate()  # évalue le modèle sur le dataset de test
#evaluate() est une méthode fournie par la classe Trainer de Hugging Face.
#Trainer fait automatiquement : Passe le modèle en mode évaluation (model.eval())

#Calcule :  Validation loss (automatique), Les métriques définies dans compute_metrics, Retourne un dictionnaire de résultats
print("Résultats finaux sur balanced dataset de test :")
print(metrics)

Résultats finaux sur balanced dataset de test :
{'eval_loss': 0.04409154877066612, 'eval_precision': 0.9990294033685413, 'eval_recall': 0.9864141157900671, 'eval_f1': 0.9926816815113179, 'eval_runtime': 64.7057, 'eval_samples_per_second': 546.737, 'eval_steps_per_second': 17.093, 'epoch': 1.0}


Lancer l'entraînement 90-10 dataset+ calcul les metric

In [27]:
#change output_dir pour chaque experience +Toujours construire un modèle neuf
training_args = TrainingArguments(
    output_dir="./results/90-10-dataset", # dossier pour sauvegarder les checkpoints
    num_train_epochs=1, #nombre de passages sur tout le dataset
    per_device_train_batch_size=16, # batch size pour l'entraînement
    per_device_eval_batch_size=32,# batch size pour évaluation
    learning_rate=5e-5,
    save_strategy="epoch", # sauvegarder modèle à chaque epoch
    eval_strategy="epoch", # Added evaluation strategy , évaluer à chaque fin d'epoch
    fp16=True  # activer float16 si GPU compatible
)

In [28]:
model_90_10=build_lora_roberta()
trainer_90_10 = Trainer(
   model= model_90_10,
    args=training_args,
    train_dataset=train_90_10,
    eval_dataset=test_90_10,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_90_10.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2001469779.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_90_10 = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.020400,0.019273,0.994937,0.980050,0.987437


TrainOutput(global_step=8845, training_loss=0.02585402796536667, metrics={'train_runtime': 791.6523, 'train_samples_per_second': 178.746, 'train_steps_per_second': 11.173, 'total_flos': 9404282506644480.0, 'train_loss': 0.02585402796536667, 'epoch': 1.0})

In [29]:
metrics = trainer_90_10.evaluate()
print("Résultats finaux sur 90-10 dataset de test :")
print(metrics)

Résultats finaux sur 90-10 dataset de test :
{'eval_loss': 0.019273225218057632, 'eval_precision': 0.9949367088607595, 'eval_recall': 0.9800498753117207, 'eval_f1': 0.9874371859296482, 'eval_runtime': 64.5152, 'eval_samples_per_second': 548.351, 'eval_steps_per_second': 17.143, 'epoch': 1.0}


Lancer l'entraînement 10-90 dataset+ calcul les metric

In [30]:
training_args = TrainingArguments(
    output_dir="./results/10-90-dataset", # dossier pour sauvegarder les checkpoints
    num_train_epochs=1, #nombre de passages sur tout le dataset
    per_device_train_batch_size=16, # batch size pour l'entraînement
    per_device_eval_batch_size=32,# batch size pour évaluation
    learning_rate=5e-5,
    save_strategy="epoch", # sauvegarder modèle à chaque epoch
    eval_strategy="epoch", # Added evaluation strategy , évaluer à chaque fin d'epoch
    fp16=True  # activer float16 si GPU compatible
)

In [31]:
model_10_90=build_lora_roberta()
trainer_10_90 = Trainer(
    model=model_10_90,
    args=training_args,
    train_dataset=train_10_90,
    eval_dataset=test_10_90,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_10_90.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-610124807.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_10_90 = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.019900,0.031882,0.999399,0.991587,0.995478


TrainOutput(global_step=8845, training_loss=0.03781309812469224, metrics={'train_runtime': 773.6713, 'train_samples_per_second': 182.901, 'train_steps_per_second': 11.433, 'total_flos': 9404282506644480.0, 'train_loss': 0.03781309812469224, 'epoch': 1.0})

In [32]:
metrics = trainer_10_90.evaluate()
print("Résultats finaux sur 10-90 dataset de test :")
print(metrics)

Résultats finaux sur 10-90 dataset de test :
{'eval_loss': 0.03188216686248779, 'eval_precision': 0.9993988673395134, 'eval_recall': 0.9915871421396283, 'eval_f1': 0.9954776799079779, 'eval_runtime': 64.7438, 'eval_samples_per_second': 546.415, 'eval_steps_per_second': 17.083, 'epoch': 1.0}
